In [2]:
import pandas as pd

df = pd.read_csv("./data/Trade_DetailedTradeMatrix.csv", encoding='latin-1')

This is the part where we analyze the data.

In [49]:
import duckdb

def dependency(country, year, item):
    query = '''
    select Partner_Countries,
        round(sum(Value) /
        (
            select
                sum(Value)
            from df
            where Year = {year}
            and Reporter_Countries = '{country}'
            and Item = '{item}'
            and Element = 'Import Quantity'
        ), 2) as dep_ratio
    from df
    where Year = {year}
    and Reporter_Countries = '{country}'
    and Element = 'Import Quantity'
    and Item = '{item}'
    and Value > 0
    group by Partner_Countries
    order by dep_ratio desc; 
'''.format(item = item, country = country, year = year)
    
    return duckdb.sql(query).fetchdf()

dependency('United Arab Emirates', 2019, 'Rice')

,Partner_Countries,dep_ratio
0,Pakistan,0.57
1,India,0.36
2,United States of America,0.03
3,Philippines,0.01
4,Sri Lanka,0.01
5,Thailand,0.01
6,Italy,0.00
7,Belgium,0.00
8,United Kingdom of Great Britain and Northern I...,0.00
9,Canada,0.00


In [45]:
duckdb.query('''
drop table if exists exp;
drop table if exists imp;

create table exp as
select Partner_Countries,
            sum(Value) as trade
        from df
        where Year = 2019
        and Reporter_Countries = 'India'
        and Element = 'Export Quantity'
        and Item = 'Rice'
        and Value > 0
        group by Partner_Countries
        order by trade desc;

create table imp as
select Reporter_Countries,
            sum(Value) as import_quantity
        from df
        where Year = 2019
        and Element = 'Import Quantity'
        and Item = 'Rice'
        and Value > 0
        group by Reporter_Countries
        order by import_quantity desc;

select Reporter_Countries, trade, import_quantity, round(trade / import_quantity, 3) as dep
from exp
left join imp
on imp.Reporter_Countries = exp.Partner_Countries
where (dep between 0 and 1)
order by dep desc;
''').fetchdf()

,Reporter_Countries,trade,import_quantity,dep
0,Bhutan,76.00,76.00,1.000
1,Nepal,234406.94,265486.18,0.883
2,Bangladesh,4430.02,5101.79,0.868
3,Qatar,41.50,51.15,0.811
4,Bahrain,110.07,143.98,0.764
5,Australia,96.32,166.24,0.579
6,Egypt,0.28,0.53,0.528
7,United States of America,54.46,177.02,0.308
8,Philippines,7201.54,27643.60,0.261
9,United Arab Emirates,184.54,1010.61,0.183
